In [55]:
!ls data/han/자동

가평_2016.xlsx	서상_2016.xlsx	의암호_2016.xlsx  화천_2016.xlsx
가평_2017.xlsx	서상_2017.xlsx	의암호_2017.xlsx  화천_2017.xlsx
가평_2018.xlsx	서상_2018.xlsx	의암호_2018.xlsx  화천_2018.xlsx
가평_2019.xlsx	서상_2019.xlsx	의암호_2019.xlsx  화천_2019.xlsx


In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
folder = 'data/han/자동'
file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
              ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

# file_names = [['자동_해평_2016.xlsx','자동_해평_2017.xlsx','자동_해평_2018.xlsx', '자동_해평_2019.xlsx'], 
#               ['자동_도개_2016.xlsx','자동_도개_2017.xlsx','자동_도개_2018.xlsx', '자동_도개_2019.xlsx'],
# #               ['자동_감천_2016.xlsx','자동_감천_2017.xlsx','자동_감천_2018.xlsx', '자동_감천_2019.xlsx'],
#               ['자동_신암_2016.xlsx','자동_신암_2017.xlsx','자동_신암_2018.xlsx', '자동_신암_2019.xlsx'],
#               ['보_구미보_2016.xlsx','보_구미보_2017.xlsx','보_구미보_2018.xlsx', '보_구미보_2019.xlsx']]

# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]


day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
#         df_tmp = pd.read_excel(path)
#         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
#         print(df_tmp.head)
#         df_loc.append(df_tmp)
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].iloc[:,2:11])
#     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
#     df.append(df_full[loc].iloc[:, 2:11])
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)
    print(df[loc].shape[0])
    rows = df[loc].shape[0]
    percentage = round(rows * 0.7)
    df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

가평_2016.xlsx
가평_2017.xlsx
가평_2018.xlsx
가평_2019.xlsx
35018
의암호_2016.xlsx
의암호_2017.xlsx
의암호_2018.xlsx
의암호_2019.xlsx
35050
서상_2016.xlsx
서상_2017.xlsx
서상_2018.xlsx
서상_2019.xlsx
35055


In [67]:
df[0].shape

(35018, 13)

In [5]:
# train_mean = []
# train_std = []
# for i in range(len(file_names)):
#     mean_tmp = df[i].mean()
#     std_tmp = df[i].std()
#     df[i] = (df[i]-mean_tmp)/std_tmp
#     train_mean.append(mean_tmp)
#     train_std.append(std_tmp)

In [8]:
# train_mean, train_std

([수온          1.606050e+01
  수소이온농도      7.773786e+00
  전기전도도       2.553592e+02
  용존산소        9.983449e+00
  탁도          6.842271e+00
  총유기탄소       2.715657e+00
  클로로필-a      1.520059e+01
  페놀          1.668259e-05
  미생물_독성지수    1.484785e-02
  Day sin     6.037108e-15
  Day cos     2.747555e-15
  Year sin   -5.681266e-08
  Year cos    2.053380e-05
  dtype: float64,
  수온          1.636421e+01
  수소이온농도      8.049377e+00
  전기전도도       2.523046e+02
  용존산소        1.052238e+01
  탁도          6.934259e+00
  총유기탄소       2.867972e+00
  총질소         2.711011e+00
  총인          1.951296e-02
  클로로필-a      1.833810e+01
  미생물_독성지수    2.509576e-03
  Day sin     6.037108e-15
  Day cos     2.747555e-15
  Year sin   -5.681266e-08
  Year cos    2.053380e-05
  dtype: float64],
 [수온           8.778589
  수소이온농도       0.435443
  전기전도도       40.671507
  용존산소         2.564432
  탁도          12.754093
  총유기탄소        0.743784
  클로로필-a      12.351946
  페놀           0.000798
  미생물_독성지수     2.146404
  Day sin      0.7

In [6]:
# normalize data

df_all = pd.concat(df)
df_all


train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std
#     df[i] = df[i].dropna(axis='columns', how = 'all')abs
#     df[i] = df[i].dropna(axis='columns',  thresh=percentage)
    

In [65]:
8760*4

35040

In [7]:
df[1]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.441552,-0.131958,0.720602,0.295279,NaN,0.001303,NaN,NaN,-0.238574,0.000380,1.414118,-0.003927,1.413850
1,-1.441552,-0.131958,0.720602,0.295279,NaN,0.001303,NaN,NaN,-0.144148,0.366417,1.365932,-0.002913,1.413853
2,-1.441552,-0.131958,0.720602,0.247012,NaN,0.001303,NaN,NaN,-0.196607,0.707510,1.224657,-0.001899,1.413854
3,-1.455128,-0.131958,0.720602,0.247012,NaN,0.001303,NaN,NaN,-0.343491,1.000413,0.999922,-0.000885,1.413855
4,-1.455128,-0.131958,0.720602,0.295279,NaN,0.001303,NaN,NaN,NaN,1.225165,0.707041,0.000129,1.413855
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35045,-1.210757,0.696788,-0.532928,0.874480,-0.180385,-0.278518,-0.471466,-0.59571,NaN,-1.365690,0.365974,-0.008266,1.413832
35046,-1.210757,0.696788,-0.459191,0.922746,-0.180385,-0.278518,-0.494075,-0.59571,NaN,-1.224405,0.707041,-0.007252,1.413838
35047,-1.224334,0.696788,-0.459191,0.922746,-0.180385,-0.278518,-0.403640,-0.72178,NaN,-0.999653,0.999922,-0.006238,1.413842
35048,-1.224334,0.696788,-0.385454,0.922746,-0.180385,-0.278518,-0.428761,-0.59571,NaN,-0.706750,1.224657,-0.005224,1.413846


In [8]:
train_mean, train_std

(수온           13.718260
 수소이온농도        7.363691
 전기전도도       106.227412
 용존산소         10.488234
 탁도            2.190726
 총유기탄소         1.599534
 총질소           1.617681
 총인            0.007725
 클로로필-a        8.673920
 Day sin      -0.000269
 Day cos       0.000026
 Year sin      0.000110
 Year cos      0.000146
 dtype: float64,
 수온           7.365852
 수소이온농도       0.482657
 전기전도도       13.561701
 용존산소         2.071821
 탁도           8.264139
 총유기탄소        0.357371
 총질소          0.398078
 총인           0.007932
 클로로필-a       9.531307
 Day sin      0.707084
 Day cos      0.707136
 Year sin     0.707038
 Year cos     0.707183
 dtype: float64)

In [10]:
df[0]

,Day cos,Day sin,Year cos,Year sin,미생물_독성지수,수소이온농도,수온,용존산소,전기전도도,총유기탄소,클로로필-a,탁도,페놀
0,1.414203,-1.999581e-12,1.414155,-0.003771,NaN,-0.974057,-1.296343,0.607528,2.019218,-1.148407,-0.529166,-0.242254,NaN
1,1.366016,3.660228e-01,1.414157,-0.002757,NaN,-0.974057,-1.296343,0.607528,1.994322,-1.032192,-0.521539,-0.250365,NaN
2,1.224736,7.071017e-01,1.414159,-0.001743,NaN,-0.974057,-1.296343,0.568510,1.994322,-1.032192,-0.529166,-0.250365,NaN
3,0.999993,9.999929e-01,1.414160,-0.000730,NaN,-0.974057,-1.296343,0.568510,1.994322,-1.148407,-0.506285,-0.250365,NaN
4,0.707102,1.224736e+00,1.414160,0.000284,NaN,-0.974057,-1.296343,0.568510,2.044113,-1.032192,-0.521539,-0.250365,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,0.366023,-1.366016e+00,1.414137,-0.008109,-0.194338,-0.205486,-1.329562,0.997712,0.152046,-0.218688,0.058116,-0.436921,NaN
35060,0.707102,-1.224736e+00,1.414142,-0.007096,-0.257362,-0.205486,-1.329562,0.997712,0.176942,-0.218688,0.058116,-0.436921,NaN
35061,0.999993,-9.999929e-01,1.414147,-0.006082,-0.068290,-0.205486,-1.340635,0.997712,0.176942,-0.218688,0.058116,-0.436921,NaN
35062,1.224736,-7.071017e-01,1.414151,-0.005068,-0.194338,-0.205486,-1.340635,0.997712,0.176942,-0.218688,0.058116,-0.436921,NaN


In [9]:
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [10]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [11]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [12]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [13]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [14]:
# not used
# original make_dataset code
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

#WindowGenerator.make_dataset = make_dataset

In [15]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [16]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [17]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [18]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [19]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
   
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}
  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters

In [20]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
        print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [21]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)

[  0   0   0 ... 942 942 942]
[[    0    22     3     0]
 [    3     9     1    22]
 [    4    14     2    31]
 ...
 [ 3296     6     1 13307]
 [ 3297    13     3 13313]
 [ 3300    13    11 13326]]
miss_data file saved


In [22]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [23]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 alpha=100.):
        'Initialization'
        window_size = input_width

        
        
#         self.data = interpolate(data_list, max_gap=fill_no)
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)
    
    
#         isnan = np.isnan(data_list)
#         isany = np.any(isnan, axis=1)
#         #shift same as pd.shift(isany, fill_value=True)
#         shifted = np.roll(isany, 1)
#         shifted[0] = True # set to nan

#         start_seq = ((isany == False) & (shifted == True)).astype(int)
#         cum = start_seq.cumsum()
#         cum += last_cum
#         last_cum = np.max(cum)
#         cum[isany] = 0
#         cums.append(cum)

        
        # normlize for spam
        if normalize:
            self.data, norm_param = normalization(self.data)
        #print(norm_param)
        
        # Define mask matrix
        if miss_pattern is None:
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            self.miss = MissData(load_dir='save')
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [24]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.251486,0.282415,1.457973,0.150479,-0.192485,-0.558340,1.960215,-0.21750,-0.312016,0.000380,1.414118,-0.003927,1.413850
1,-1.278638,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.191326,-0.59571,-0.343491,0.366417,1.365932,-0.002913,1.413853
2,-1.265062,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.329489,-0.59571,-0.343491,0.707510,1.224657,-0.001899,1.413854
3,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.171229,-0.72178,-0.343491,1.000413,0.999922,-0.000885,1.413855
4,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.098379,-0.59571,-0.343491,1.225165,0.707041,0.000129,1.413855
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35013,-1.373671,-0.546331,0.646865,1.019280,-0.131983,0.560945,0.252009,-0.72178,1.471580,-1.365690,0.365974,-0.008266,1.413832
35014,-1.387248,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.272106,-0.84785,1.387646,-1.224405,0.707041,-0.007252,1.413838
35015,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.302251,-0.72178,1.324696,-0.999653,0.999922,-0.006238,1.413842
35016,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.264570,-0.72178,1.345679,-0.706750,1.224657,-0.005224,1.413846


In [25]:
dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

In [28]:
rows = df[0].shape[0]
rows

35018

In [29]:
df[0]


,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.251486,0.282415,1.457973,0.150479,-0.192485,-0.558340,1.960215,-0.21750,-0.312016,0.000380,1.414118,-0.003927,1.413850
1,-1.278638,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.191326,-0.59571,-0.343491,0.366417,1.365932,-0.002913,1.413853
2,-1.265062,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.329489,-0.59571,-0.343491,0.707510,1.224657,-0.001899,1.413854
3,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.171229,-0.72178,-0.343491,1.000413,0.999922,-0.000885,1.413855
4,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.098379,-0.59571,-0.343491,1.225165,0.707041,0.000129,1.413855
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35013,-1.373671,-0.546331,0.646865,1.019280,-0.131983,0.560945,0.252009,-0.72178,1.471580,-1.365690,0.365974,-0.008266,1.413832
35014,-1.387248,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.272106,-0.84785,1.387646,-1.224405,0.707041,-0.007252,1.413838
35015,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.302251,-0.72178,1.324696,-0.999653,0.999922,-0.006238,1.413842
35016,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.264570,-0.72178,1.345679,-0.706750,1.224657,-0.005224,1.413846


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35050,4.0,7.4,111.0,12.2,0.4,1.6,1.584,NaN,13.7,-9.659258e-01,0.258819,-0.005734,0.999984
35051,3.5,7.4,112.0,12.3,0.4,1.6,1.499,NaN,13.1,-8.660254e-01,0.500000,-0.005017,0.999987
35052,3.1,7.4,112.0,12.4,0.4,1.7,1.589,NaN,12.7,-7.071068e-01,0.707107,-0.004301,0.999991
35053,2.8,7.4,113.0,12.5,0.4,1.6,1.488,NaN,12.6,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
    wide_window = WindowGenerator(
        input_width=24*5, label_width=24*5, shift=0,
        #label_columns=['T (degC)']
    )

    wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

In [32]:
wide_window.example[0].shape

TensorShape([128, 120, 13])

In [33]:
df_all

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35050,4.0,7.4,111.0,12.2,0.4,1.6,1.584,NaN,13.7,-9.659258e-01,0.258819,-0.005734,0.999984
35051,3.5,7.4,112.0,12.3,0.4,1.6,1.499,NaN,13.1,-8.660254e-01,0.500000,-0.005017,0.999987
35052,3.1,7.4,112.0,12.4,0.4,1.7,1.589,NaN,12.7,-7.071068e-01,0.707107,-0.004301,0.999991
35053,2.8,7.4,113.0,12.5,0.4,1.6,1.488,NaN,12.6,-5.000000e-01,0.866025,-0.003584,0.999994


In [34]:
wide_window.plot(plot_col='총유기탄소')
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  7218


In [35]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
plt.figure(figsize=(9,10))
n = wide_window.dg.data_m.shape[0]
train = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(wide_window.dg.data_m[i*train:(i+1)*train, 0:7], aspect='auto')
    plt.yticks([])
#plt.imshow(wide_window.dg.data[0:100])
#plt.imshow(wide_window.dg.data_m[800:900], aspect='auto')
#print(wide_window.dg.data[0:50])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [37]:
val_performance = {}
performance = {}

In [38]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.251486,0.282415,1.457973,0.150479,-0.192485,-0.558340,1.960215,-0.21750,-0.312016,0.000380,1.414118,-0.003927,1.413850
1,-1.278638,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.191326,-0.59571,-0.343491,0.366417,1.365932,-0.002913,1.413853
2,-1.265062,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.329489,-0.59571,-0.343491,0.707510,1.224657,-0.001899,1.413854
3,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.171229,-0.72178,-0.343491,1.000413,0.999922,-0.000885,1.413855
4,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.098379,-0.59571,-0.343491,1.225165,0.707041,0.000129,1.413855
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35013,-1.373671,-0.546331,0.646865,1.019280,-0.131983,0.560945,0.252009,-0.72178,1.471580,-1.365690,0.365974,-0.008266,1.413832
35014,-1.387248,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.272106,-0.84785,1.387646,-1.224405,0.707041,-0.007252,1.413838
35015,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.302251,-0.72178,1.324696,-0.999653,0.999922,-0.006238,1.413842
35016,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.264570,-0.72178,1.345679,-0.706750,1.224657,-0.005224,1.413846


In [39]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [40]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [41]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

gain.save(save_dir='save')

Epoch 1/2000
1/1 [==============================] - 1s 961ms/step - gen_loss: 106.9922 - disc_loss: 0.7265 - rmse: 0.9879 - val_loss: 0.8674
Epoch 2/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 87.0969 - disc_loss: 0.5232 - rmse: 0.8936 - val_loss: 0.8815
Epoch 3/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 75.5065 - disc_loss: 0.5405 - rmse: 0.9260 - val_loss: 0.7393
Epoch 4/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 61.9547 - disc_loss: 0.3967 - rmse: 0.7400 - val_loss: 0.7359
Epoch 5/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 50.6245 - disc_loss: 0.4075 - rmse: 0.6641 - val_loss: 0.6641
Epoch 6/2000
1/1 [==============================] - 0s 164ms/step - gen_loss: 47.1815 - disc_loss: 0.4078 - rmse: 0.6803 - val_loss: 0.6167
Epoch 7/2000
1/1 [==============================] - 0s 157ms/step - gen_loss: 47.7652 - disc_loss: 0.4098 - rmse: 0.7664 - val_loss: 0.6789
Epoch 8/2000
1/1 [=

1/1 [==============================] - 0s 157ms/step - gen_loss: 6.8122 - disc_loss: 0.3133 - rmse: 0.2758 - val_loss: 0.2834
Epoch 60/2000
1/1 [==============================] - 0s 159ms/step - gen_loss: 6.4544 - disc_loss: 0.3174 - rmse: 0.3338 - val_loss: 0.2698
Epoch 61/2000
1/1 [==============================] - 0s 155ms/step - gen_loss: 5.0798 - disc_loss: 0.3100 - rmse: 0.2772 - val_loss: 0.2820
Epoch 62/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 6.1020 - disc_loss: 0.3110 - rmse: 0.3233 - val_loss: 0.2762
Epoch 63/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 6.3034 - disc_loss: 0.3108 - rmse: 0.4068 - val_loss: 0.2615
Epoch 64/2000
1/1 [==============================] - 0s 157ms/step - gen_loss: 5.3368 - disc_loss: 0.3218 - rmse: 0.3190 - val_loss: 0.3072
Epoch 65/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 6.8500 - disc_loss: 0.3060 - rmse: 0.2946 - val_loss: 0.3033
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 156ms/step - gen_loss: 5.8241 - disc_loss: 0.2677 - rmse: 0.2552 - val_loss: 0.2326
Epoch 118/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 4.6564 - disc_loss: 0.2597 - rmse: 0.2538 - val_loss: 0.2507
Epoch 119/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 4.3493 - disc_loss: 0.2679 - rmse: 0.2410 - val_loss: 0.2573
Epoch 120/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 4.3380 - disc_loss: 0.2547 - rmse: 0.5202 - val_loss: 0.2406
Epoch 121/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 4.3446 - disc_loss: 0.2594 - rmse: 0.2480 - val_loss: 0.2597
Epoch 122/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 5.0952 - disc_loss: 0.2562 - rmse: 0.2567 - val_loss: 0.2405
Epoch 123/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 5.5440 - disc_loss: 0.2559 - rmse: 0.2513 - val_loss: 0.2679
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 154ms/step - gen_loss: 5.3223 - disc_loss: 0.2160 - rmse: 0.3474 - val_loss: 0.2160
Epoch 176/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 4.5387 - disc_loss: 0.2245 - rmse: 0.2324 - val_loss: 0.2288
Epoch 177/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 4.8380 - disc_loss: 0.2218 - rmse: 0.2360 - val_loss: 0.2188
Epoch 178/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 5.1470 - disc_loss: 0.2184 - rmse: 0.2574 - val_loss: 0.2544
Epoch 179/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 4.7616 - disc_loss: 0.2150 - rmse: 0.2269 - val_loss: 0.2232
Epoch 180/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 5.4033 - disc_loss: 0.2166 - rmse: 0.2932 - val_loss: 0.2398
Epoch 181/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 4.1926 - disc_loss: 0.2151 - rmse: 0.2547 - val_loss: 0.2296
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 153ms/step - gen_loss: 3.5292 - disc_loss: 0.2009 - rmse: 0.5068 - val_loss: 0.2357
Epoch 234/2000
1/1 [==============================] - 0s 155ms/step - gen_loss: 3.7204 - disc_loss: 0.1945 - rmse: 0.2271 - val_loss: 0.2427
Epoch 235/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 3.2598 - disc_loss: 0.1970 - rmse: 0.2407 - val_loss: 0.2427
Epoch 236/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 3.4664 - disc_loss: 0.2006 - rmse: 0.3920 - val_loss: 0.2492
Epoch 237/2000
1/1 [==============================] - 0s 159ms/step - gen_loss: 3.1332 - disc_loss: 0.1939 - rmse: 0.2254 - val_loss: 0.2203
Epoch 238/2000
1/1 [==============================] - 0s 164ms/step - gen_loss: 3.5321 - disc_loss: 0.1934 - rmse: 0.5084 - val_loss: 0.2400
Epoch 239/2000
1/1 [==============================] - 0s 162ms/step - gen_loss: 2.8724 - disc_loss: 0.1901 - rmse: 0.2034 - val_loss: 0.2240
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 154ms/step - gen_loss: 1.9106 - disc_loss: 0.1095 - rmse: 0.1669 - val_loss: 0.4782
Epoch 1730/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 2.3558 - disc_loss: 0.1081 - rmse: 0.1900 - val_loss: 0.1517
Epoch 1731/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 2.5946 - disc_loss: 0.1097 - rmse: 0.1954 - val_loss: 0.3964
Epoch 1732/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 2.0420 - disc_loss: 0.1115 - rmse: 0.1748 - val_loss: 0.1618
Epoch 1733/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 1.9835 - disc_loss: 0.1120 - rmse: 0.1644 - val_loss: 0.3534
Epoch 1734/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 2.0463 - disc_loss: 0.1085 - rmse: 0.1824 - val_loss: 0.2232
Epoch 1735/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 1.8765 - disc_loss: 0.1100 - rmse: 0.1620 - val_loss: 0.1652
Epoch 1736/2000
1/1 [=

1/1 [==============================] - 0s 153ms/step - gen_loss: 1.8997 - disc_loss: 0.1149 - rmse: 0.1528 - val_loss: 0.1615
Epoch 1787/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 2.3502 - disc_loss: 0.1131 - rmse: 0.1710 - val_loss: 0.1795
Epoch 1788/2000
1/1 [==============================] - 0s 155ms/step - gen_loss: 2.3074 - disc_loss: 0.1161 - rmse: 0.1555 - val_loss: 0.1713
Epoch 1789/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 1.8340 - disc_loss: 0.1156 - rmse: 0.1508 - val_loss: 0.1779
Epoch 1790/2000
1/1 [==============================] - 0s 158ms/step - gen_loss: 2.0724 - disc_loss: 0.1167 - rmse: 0.1560 - val_loss: 0.1689
Epoch 1791/2000
1/1 [==============================] - 0s 158ms/step - gen_loss: 1.7783 - disc_loss: 0.1155 - rmse: 0.1539 - val_loss: 0.1700
Epoch 1792/2000
1/1 [==============================] - 0s 157ms/step - gen_loss: 2.4780 - disc_loss: 0.1155 - rmse: 0.1637 - val_loss: 0.1712
Epoch 1793/2000
1/1 [=

1/1 [==============================] - 0s 156ms/step - gen_loss: 1.7923 - disc_loss: 0.1118 - rmse: 0.1555 - val_loss: 0.2780
Epoch 1844/2000
1/1 [==============================] - 0s 158ms/step - gen_loss: 1.9145 - disc_loss: 0.1151 - rmse: 0.1558 - val_loss: 0.1774
Epoch 1845/2000
1/1 [==============================] - 0s 158ms/step - gen_loss: 1.8172 - disc_loss: 0.1175 - rmse: 0.1399 - val_loss: 0.2183
Epoch 1846/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 2.6411 - disc_loss: 0.1119 - rmse: 0.1762 - val_loss: 0.1736
Epoch 1847/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 1.8400 - disc_loss: 0.1131 - rmse: 0.1578 - val_loss: 0.1753
Epoch 1848/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 1.8082 - disc_loss: 0.1119 - rmse: 0.1591 - val_loss: 0.1535
Epoch 1849/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 1.7994 - disc_loss: 0.1132 - rmse: 0.1434 - val_loss: 0.1505
Epoch 1850/2000
1/1 [=

1/1 [==============================] - 0s 154ms/step - gen_loss: 2.5274 - disc_loss: 0.1158 - rmse: 0.1869 - val_loss: 0.1691
Epoch 1901/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 1.7287 - disc_loss: 0.1180 - rmse: 0.1601 - val_loss: 0.1739
Epoch 1902/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 1.8964 - disc_loss: 0.1148 - rmse: 0.1547 - val_loss: 0.1704
Epoch 1903/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 1.7990 - disc_loss: 0.1164 - rmse: 0.1558 - val_loss: 0.1669
Epoch 1904/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 2.1895 - disc_loss: 0.1095 - rmse: 0.1554 - val_loss: 0.1723
Epoch 1905/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 1.9537 - disc_loss: 0.1140 - rmse: 0.1588 - val_loss: 0.1779
Epoch 1906/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 1.8319 - disc_loss: 0.1156 - rmse: 0.1499 - val_loss: 0.1719
Epoch 1907/2000
1/1 [=

1/1 [==============================] - 0s 152ms/step - gen_loss: 2.1366 - disc_loss: 0.1121 - rmse: 0.1495 - val_loss: 0.1659
Epoch 1958/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 1.6754 - disc_loss: 0.1117 - rmse: 0.1439 - val_loss: 0.1627
Epoch 1959/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 1.9178 - disc_loss: 0.1153 - rmse: 0.1611 - val_loss: 0.1668
Epoch 1960/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 2.0071 - disc_loss: 0.1122 - rmse: 0.1683 - val_loss: 0.1641
Epoch 1961/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 1.8907 - disc_loss: 0.1141 - rmse: 0.1525 - val_loss: 0.1737
Epoch 1962/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 1.9386 - disc_loss: 0.1129 - rmse: 0.1459 - val_loss: 0.1757
Epoch 1963/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 1.7809 - disc_loss: 0.1148 - rmse: 0.1659 - val_loss: 0.1586
Epoch 1964/2000
1/1 [=

**학습 loss history 출력**

In [42]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

성능 측정

In [43]:
gain.evaluate(wide_window.test.repeat(), steps=100)

100/100 [==============================] - 3s 26ms/step - loss: 0.1738


0.17376808822155

샘플 prediction 출력

In [44]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
norm_df = pd.concat(df,axis=0)

In [46]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105123
(120, 13)
120
x.shape = (105120, 13)
x_reshape.shape = (876, 120, 13)
182701
y_true.shape= (105120, 13)
182701


(1, 120, 13)

In [47]:
# zero loss is normal because there is no ground truth in the real dataset
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 5ms/step - loss: 0.0000e+00


0.0

array([[-1.31689696, -0.85840298,  1.95814725, ...,         nan,
                nan,         nan],
       [-1.31689696, -0.85840298,  1.93356002, ...,         nan,
                nan,         nan],
       [-1.31689696, -0.85840298,  1.93356002, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
         0.30323712,  0.30301428],
       [        nan,         nan,         nan, ...,         nan,
         0.30323712,  0.30160336],
       [        nan,         nan,         nan, ...,         nan,
         0.30323712,  0.30160336]])

In [48]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [49]:
print(total_n)
print(n)
print(total_n-n)

105123
105120
3


In [50]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 13) (120, 13)
(105123, 13)


In [51]:
y_pred[~np.isnan(data)] = np.nan

In [52]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105123
unit_shape= (120, 13)
120
n= 105120
(35040, 9)
(35040, 9)


In [103]:
gans[1].shape

(35040, 9)

In [104]:
df[1].shape

(35050, 13)

In [91]:
dd = pd.DataFrame(gans[0])
dd.to_excel('gain1.xlsx', index=False)

## 예측 

In [ ]:
total_no = oris[0].shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gans[0][train_slice])
val_df = pd.DataFrame(gans[0][val_slice])
test_df = pd.DataFrame(gans[0][test_slice])

train_ori_df = pd.DataFrame(oris[0][train_slice])
val_ori_df = pd.DataFrame(oris[0][val_slice])
test_ori_df = pd.DataFrame(oris[0][test_slice])

num_features = train_df.shape[1]
out_num_features = num_features

In [ ]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
out_features = [6]
out_num_features = len(out_features)
out_num_features


In [ ]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [ ]:
OUT_STEPS = 24*3
wdg = WaterDataGenerator(train_df,
                         batch_size=128,
                         input_width = 24*7,
                         label_width = OUT_STEPS,
                         shift = OUT_STEPS,
                         out_num_features = out_num_features
                        )

In [ ]:
train_df

In [ ]:
it = iter(wdg)

In [ ]:
x,y = next(it)
x.shape, y.shape

In [ ]:
def make_dataset_water(self, data):
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [ ]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

In [ ]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
multi_window

In [ ]:
MAX_EPOCHS = 400

def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

**Baseline model**

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])


In [ ]:
multi_window.plot2(last_baseline, plot_col=out_features[0])

**Linear**

In [ ]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

In [ ]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_linear_model, plot_col=out_features[0])

**Elman_RNN**

In [ ]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(elman_model, plot_col=5)

In [ ]:
multi_window.plot2(elman_model, plot_col=out_features[0])

**GRU**

In [ ]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
MAX_EPOCHS = 450

In [ ]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(gru_model, plot_col=3)

In [ ]:
multi_window.plot2(gru_model, plot_col=out_features[0])

**LSTM**

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_lstm_model, plot_col=out_features[0])

In [ ]:
multi_window.plot2(multi_lstm_model, plot_col=5)

**CNN model**

In [ ]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
MAX_EPOCHS = 400
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_conv_model, plot_col=out_features[0])

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

**Autoregressive model**

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)
    
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)
  #print('x =', x)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  predictions = tf.keras.layers.Lambda(lambda x: x[:, :, out_features[0]:(out_features[0]+len(out_features))])(predictions)
  return predictions

FeedBack.call = call

print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
history = compile_and_fit(feedback_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)

print('val performance =', multi_val_performance['AR LSTM'])
print('test performance = ', multi_performance['AR LSTM'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(feedback_model, plot_col=out_features[0])

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()